In [28]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F

from pyspark.ml.feature import Tokenizer

import os
import sys
import pandas as pd
import json

# import nltk
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize

In [2]:
conf = SparkConf()

spark = spark = SparkSession \
    .builder \
    .appName("Lab2") \
    .config(conf=conf) \
    .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/20 22:26:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
candidatePath = 'candidate.json'
queryPath = 'query.json'
stopwordPath = 'stopwords.txt'

In [49]:
candidates = spark.read.option("multiline", "true").json(candidatePath)
candidates = candidates.withColumn('abstract', F.trim('abstract'))

queries = spark.read.option("multiline", "true").json(queryPath)
queries = queries.withColumn('abstract', F.trim('abstract'))

with open(stopwordPath, 'r') as f:
    stopwords = f.read().splitlines()

In [19]:
def test(row):
    abstract = row['abstract']
    sentences = sent_tokenize(abstract)
    return Row(abstract=abstract, sentences=sentences)

In [76]:
def list2array(row):
    abstract = row['abstract']
    final_tokens = tuple(row['final_tokens'])
    id = row['id']
    return Row(id=id, abstract=abstract, tokens=final_tokens)

In [44]:
tokenizer = Tokenizer(outputCol="tokens")
tokenizer.setInputCol('abstract')

Tokenizer_ab4fbb977a84

In [45]:
candidates_tokenized = tokenizer.transform(candidates)
candidates_tokenized = candidates_tokenized.withColumn('final_tokens', F.array_except('tokens', F.lit(stopwords)))

In [46]:
candidates_tokenized.limit(5).toPandas()

,abstract,id,tokens,final_tokens
0,The $3x+1$ Problem asks if whether for every n...,1504.03040,"[the, $3x+1$, problem, asks, if, whether, for,...","[$3x+1$, problem, asks, natural, number, $n$,,..."
1,We characterize the terahertz detection mechan...,1203.6290,"[we, characterize, the, terahertz, detection, ...","[characterize, terahertz, detection, mechanism..."
2,This paper proposes LPRNet - end-to-end method...,1806.10447,"[this, paper, proposes, lprnet, -, end-to-end,...","[paper, proposes, lprnet, -, end-to-end, metho..."
3,We develop the representation of bulk fields w...,1204.0126,"[we, develop, the, representation, of, bulk, f...","[develop, representation, bulk, fields, spin, ..."
4,We present a cascade model for turbulence in w...,0707.3149,"[we, present, a, cascade, model, for, turbulen...","[cascade, model, turbulence, weakly, collision..."


In [77]:
candidates_rdd = candidates_tokenized.limit(5).rdd.map(list2array)

In [55]:
queries_tokenized = tokenizer.transform(queries)
queries_tokenized = queries_tokenized.withColumn('final_tokens', F.array_except('tokens', F.lit(stopwords)))

In [78]:
queries_rdd = queries_tokenized.limit(5).rdd.map(list2array)

In [90]:
def toList(a):
    return [a]
def append(a, b):
    a.append(b)
    return a
def extend(a, b):
    a.extend(b)
    return a
queries_rdd.cartesian(candidates_rdd).combineByKey(toList, append, extend).map(docTFIDF).collect()

[Row(a='A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the LHC, showing\nthat enhanced sensitivity to the signal can be obtained with judicious\nselection of events

In [89]:
def docTFIDF(row):
    return Row(a=row[0]['abstract'], b=len(row[1]))

In [ ]:
def calMostSimilar(row):
    query = row[0]
    docs = row[1]
    